## Prior Monte Carlo

blahblahblah

We will also use the results of the prior MC to choose the truth for our dual experiment where we see if the modeling can bracket the simulated results of an unseen realization...

In [ ]:


import os
import shutil
import warnings
warnings.filterwarnings("ignore")
warnings.filterwarnings("ignore", category=DeprecationWarning) 
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt;
import psutil

import sys
import pyemu
import flopy
assert "dependencies" in flopy.__file__
assert "dependencies" in pyemu.__file__
sys.path.insert(0,"..")

# specify the temporary working folder
t_d = os.path.join('pst_template_pmc')
# get the previously generated PEST dataset
org_t_d = os.path.join('pst_template')
if not os.path.exists(org_t_d):
    raise Exception()
if os.path.exists(t_d):
    shutil.rmtree(t_d)
shutil.copytree(org_t_d,t_d)

In [ ]:
pst = pyemu.Pst(os.path.join(t_d,"pest.pst"))

In [ ]:
pst.observation_data["weight"] = 0.0

In [ ]:
pst.pestpp_options["ies_num_reals"] = 100
pst.pestpp_options["save_binary"] = True
pst.control_data.noptmax = -1

In [ ]:
pst.write(os.path.join(t_d,"pest.pst"),version=2)

In [ ]:
num_workers=30
m_d = "master_prior_mc"

In [ ]:
pyemu.os_utils.start_workers(t_d, # the folder which contains the "template" PEST dataset
                            'pestpp-ies', #the PEST software version we want to run
                            'pest.pst', # the control file to use with PEST
                            num_workers=num_workers, #how many agents to deploy
                            worker_root='.', #where to deploy the agent directories; relative to where python is running
                            master_dir=m_d, #the manager directory
                            )

In [ ]:
pst = pyemu.Pst(os.path.join(m_d,"pest.pst"))

In [ ]:
oe = pst.ies.obsen

In [ ]:
forecasts = [f.strip() for f in pst.pestpp_options["forecasts"].split(',')]
forecasts.sort()
forecasts

In [ ]:
feas_oe = oe.loc[oe[forecasts[1]]<=80,:]
oe.shape,feas_oe.shape

In [ ]:
mx_drn = feas_oe[forecasts[0]].min()
mx_drn

In [ ]:
truth_real = feas_oe.loc[feas_oe[forecasts[0]]==mx_drn,:].index
truth_real

In [ ]:
for f in forecasts:
    ax = oe.loc[:,f].plot(kind="hist",fc="0.5",alpha=0.5,density=True)
    ax = feas_oe.loc[:,f].plot(kind="hist",ax=ax,fc="b",alpha=0.5,density=True)
    ax.set_title(f)
    plt.show()

In [ ]:
pe = pst.ies.paren0
truth_pvals = pe.loc[truth_real,:].values.flatten()
truth_pvals.shape,pst.npar

In [ ]:
org_pvals = pst.parameter_data.loc[pe.columns,"parval1"].copy()
pst.parameter_data.loc[pe.columns,"parval1"] = truth_pvals
pst.control_data.noptmax = 0
pst.write(os.path.join(m_d,"truth.pst"),version=2)

In [ ]:
pyemu.os_utils.run("pestpp-ies truth.pst",cwd=m_d)

Lets checkout the truth HK array:

In [ ]:
df = pd.read_csv(os.path.join(m_d,"mult2model_info.csv"))
kh1_df = df.loc[df.model_file.str.contains("npf_k"),:]
kh1_df
org_arr = np.loadtxt(os.path.join(m_d,kh1_df.org_file.iloc[0]))
inp_arr = np.loadtxt(os.path.join(m_d,kh1_df.model_file.iloc[0]))
mlt_arrs = [np.loadtxt(os.path.join(m_d,afile)) for afile in kh1_df.mlt_file]
arrs = [org_arr]
arrs.extend(mlt_arrs)
arrs.append(inp_arr)

mlt_arrs
names = ["org"]
names.extend([mf.split('.')[0].split('_')[-1] for mf in kh1_df.mlt_file])
names.append("MF6 input")
fig,axes = plt.subplots(1,kh1_df.shape[0]+2,figsize=(5*kh1_df.shape[0]+2,5))
for i,ax in enumerate(axes.flatten()):
    arr = np.log10(arrs[i])
    cb = ax.imshow(arr)
    plt.colorbar(cb,ax=ax, fraction=0.046, pad=0.04)
    ax.set_title(names[i],loc="left")
plt.tight_layout()    

Pretty obsvious why this one gives such high post-closure GDE flux...

In [ ]:
pst.set_res(os.path.join(m_d,"truth.base.rei"))

In [ ]:
par = pst.parameter_data
wpar = par.loc[(par.parnme.str.contains("mar")) | (par.parnme.str.contains("dewater")),:].copy()
wpar["i"] = wpar.idx1.astype(int)
wpar["j"] = wpar.idx2.astype(int)
ijs = set([(i,j) for i,j in zip(wpar.i,wpar.j)])



obs = pst.observation_data
obs["weight"] = 0.0

obs.loc[:,"obsval"] = pst.res.loc[pst.obs_names,"modelled"].values
hobs = obs.loc[obs.obsnme.str.contains("hdslay1_t1"),:].copy()
assert hobs.shape[0] > 0
hobs['ij'] = hobs.apply(lambda x: (int(x.i),int(x.j)),axis=1)
nzhobs = hobs.loc[hobs.ij.isin(ijs)]
assert len(nzhobs) == len(ijs)
obs.loc[nzhobs.obsnme,"weight"] = 3.0
drnobs = obs.loc[(obs.usecol=="drn-gde") & (obs.time=="1"),:]
assert len(drnobs) == 1
obs.loc[drnobs.obsnme[0],"weight"] = 1./(np.abs(obs.loc[drnobs.obsnme[0],"obsval"]) * 0.2)
assert pst.nnz_obs == len(ijs) + 1

In [ ]:
pst.parameter_data["parval1"] = org_pvals
pst.write(os.path.join(t_d,"pest.pst"),version=2)

We also should remove the truth from the prior ensemble to make sure we arent cheating ;)

In [ ]:
pe.drop(truth_real,inplace=True)
if "base" in pe.index:
    pe.drop("base",inplace=True)

In [ ]:
pst.pestpp_options

In [ ]:
pyemu.ParameterEnsemble(df=pe,pst=pst).to_binary(os.path.join(t_d,"prior_pe.jcb"))